In [2]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['../src', '../test', '../example'])

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux


In [3]:
import pandas as pd
from openalea.plantgl.all import Viewer
from hydroroot.display import plot as mtg_scene
from hydroroot import radius
from hydroroot.main import hydroroot_flow
from hydroroot.generator.measured_root import mtg_from_aqua_data
from shared_functions import plot

In [4]:
df = pd.read_csv('data/plant-1.txt', sep = '\t', dtype = {'order': str})
df['db'] = df['distance_from_base_(mm)'] * 1.e-3
df['lr'] = df['lateral_root_length_(mm)'] * 1.e-3

Building the MTG from the file, and some properties needed for the calculation

In [5]:
g = mtg_from_aqua_data(df, segment_length = 1.0e-4) # creating the MTG from the file with a vertex length of 0.1 mm
g = radius.ordered_radius(g, 7.0e-5, 0.7) # set the radius of the primary root to 70 um, and to 70*0.7^d to lateral of order d
g = radius.compute_relative_position(g) # calculation of properties used in the model see compute_relative_position doc

Some conductance data versus distance to tip

In [6]:
k_radial_data=([0, 0.2],[300.0,300.0])
K_axial_data=([0, 0.2],[3.0e-5,4.0e-4])

Flux and equivalent conductance calculation, for a root in an external hydroponic medium at 0.4 MPa, its base at 0.1 MPa, and with the conductances set above.

In [7]:
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, 
                            axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)

In [8]:
print(keq,jv)

0.0335903319734048 0.010077099592021441


Display the local water uptake heatmap in 3D

In [9]:
%gui qt
Viewer.display(mtg_scene(g, prop_cmap='j')) # j is the radial flux in ul/s

You may change the property to display to the hydrostatic pressure inside the xylem vessels for instance

In [10]:
Viewer.display(mtg_scene(g, prop_cmap='psi_in'))